In [1]:
import numpy as np
import pandas as pd
import sklearn.metrics
from typing import List, Optional, Dict, Set, Callable, Any
from joblib import Memory, Parallel, delayed
import tslearn
import tslearn.metrics
from tslearn.datasets import UCR_UEA_datasets
from scipy.interpolate import interp1d
from numba import njit
import pickle

from experiment_code import print_dataset_stats, run_all_kernels
from cross_validation import cv_tslearn

## Find all tslearn datasets

In [ ]:
# _datasets = [
#             'ArticularyWordRecognition', 
#             'BasicMotions', 
#             'Cricket',
#             #'ERing',
#             'Libras', 
#             'NATOPS', 
#             'RacketSports',     
#             'FingerMovements',
#             'Heartbeat',
#             'SelfRegulationSCP1', 
#             'UWaveGestureLibrary'
#             ]

# import tslearn
# UCR_UEA_datasets = tslearn.datasets.UCR_UEA_datasets()

# for dataset_name in UCR_UEA_datasets.list_multivariate_datasets():
# #for dataset_name in _datasets:
#     print("Dataset:", dataset_name)
#     dataset = UCR_UEA_datasets.load_dataset(dataset_name)
#     if dataset[0] is not None:
#         X_train, y_train, X_test, y_test = dataset
#         num_classes = len(np.unique(y_train))
#         N_train, T, d = X_train.shape
#         N_test, _, _  = X_test.shape
        
#         print("Number of Classes:", num_classes)
#         print("Dimension of path:", d)
#         print("Length:", T)
#         print("Train Size, Test Size", N_train, N_test)
#         print()
#     else:
#         print("No dataset found")
#         print()

#yes
# Dataset: ArticularyWordRecognition
# Number of Classes: 25
# Dimension of path: 9
# Length: 144
# Train Size, Test Size 275 300

# Dataset: AtrialFibrillation
# No dataset found

#yes
# Dataset: BasicMotions
# Number of Classes: 4
# Dimension of path: 6
# Length: 100
# Train Size, Test Size 40 40

# Dataset: CharacterTrajectories
# No dataset found

#yes
# Dataset: Cricket
# Number of Classes: 12
# Dimension of path: 6
# Length: 1197
# Train Size, Test Size 108 72

# Dataset: DuckDuckGeese
# No dataset found

# Dataset: EigenWorms
# Number of Classes: 5
# Dimension of path: 6
# Length: 17984
# Train Size, Test Size 128 131

#why not
# Dataset: Epilepsy
# Number of Classes: 4
# Dimension of path: 3
# Length: 206
# Train Size, Test Size 137 138

#longLength
# Dataset: EthanolConcentration
# Number of Classes: 4
# Dimension of path: 3
# Length: 1751
# Train Size, Test Size 261 263

# Dataset: ERing
# No dataset found

#big
# Dataset: FaceDetection
# Number of Classes: 2
# Dimension of path: 144
# Length: 62
# Train Size, Test Size 5890 3524

#yes
# Dataset: FingerMovements
# Number of Classes: 2
# Dimension of path: 28
# Length: 50
# Train Size, Test Size 316 100

#why not, maybe big length
# Dataset: HandMovementDirection
# Number of Classes: 4
# Dimension of path: 10
# Length: 400
# Train Size, Test Size 160 74

#smallTrain
# Dataset: Handwriting
# Number of Classes: 26
# Dimension of path: 3
# Length: 152
# Train Size, Test Size 150 850

#yes
# Dataset: Heartbeat
# Number of Classes: 2
# Dimension of path: 61
# Length: 405
# Train Size, Test Size 204 205

#big
# Dataset: InsectWingbeat
# Number of Classes: 10
# Dimension of path: 200
# Length: 22
# Train Size, Test Size 25000 25000

# Dataset: JapaneseVowels
# No dataset found

#yes
# Dataset: Libras
# Number of Classes: 15
# Dimension of path: 2
# Length: 45
# Train Size, Test Size 180 180

#TODO I SHOULD INCLUDE
# Dataset: LSST
# Number of Classes: 14
# Dimension of path: 6
# Length: 36
# Train Size, Test Size 2459 2466

#length
# Dataset: MotorImagery
# Number of Classes: 2
# Dimension of path: 64
# Length: 3000
# Train Size, Test Size 278 100

#yes
# Dataset: NATOPS
# Number of Classes: 6
# Dimension of path: 24
# Length: 51
# Train Size, Test Size 180 180

#yes
# Dataset: PenDigits
# Number of Classes: 10
# Dimension of path: 2
# Length: 8
# Train Size, Test Size 7494 3498

#TODO SHOULD INCLUDE highDim
# Dataset: PEMS-SF
# Number of Classes: 7
# Dimension of path: 963
# Length: 144
# Train Size, Test Size 267 173

#NO, dim=1, big length, large num classes
# Dataset: Phoneme
# Number of Classes: 39
# Dimension of path: 1
# Length: 1024
# Train Size, Test Size 214 1896

#yes
# Dataset: RacketSports
# Number of Classes: 4
# Dimension of path: 6
# Length: 30
# Train Size, Test Size 151 152

#yes
# Dataset: SelfRegulationSCP1
# Number of Classes: 2
# Dimension of path: 6
# Length: 896
# Train Size, Test Size 268 293

# Dataset: SelfRegulationSCP2
# Number of Classes: 2
# Dimension of path: 7
# Length: 1152
# Train Size, Test Size 200 180

# Dataset: SpokenArabicDigits
# No dataset found

#NO, long, also very small set
# Dataset: StandWalkJump
# Number of Classes: 3
# Dimension of path: 4
# Length: 2500
# Train Size, Test Size 12 15

#yes
# Dataset: UWaveGestureLibrary
# Number of Classes: 8
# Dimension of path: 3
# Length: 315
# Train Size, Test Size 120 320


# (tslearn) Cross Validation on Train

In [2]:
cv_best_models = cv_tslearn(
    dataset_names = [
        #'ArticularyWordRecognition', 
        'BasicMotions', 
        #'Cricket',
             #########'ERing', #cant find dataset
        #'Libras', 
        #'NATOPS', 
        #'RacketSports',     
        #'FingerMovements',
        #'Heartbeat',
        #'SelfRegulationSCP1', 
        #'UWaveGestureLibrary'
        ],
    kernel_names = [
        "linear",
        #"rbf",
        #"poly",
        "gak",
        "truncated sig",
        #"truncated sig rbf",
        #"truncated sig poly",
        "signature pde",
        #"signature pde rbf",
        #"signature pde poly",
        #"integral linear",
        #"integral rbf",
        #"integral poly",
        ],
        k=4,
        n_repeats=1,
        n_jobs_repeats=4,
        n_jobs_gram=1,
        verbose=False,
        )

Dataset: BasicMotions
Number of Classes: 4
Dimension of path: 6
Length: 100
Train: 40
Test: N/A


Label for linear:   0%|          | 0/4 [00:00<?, ?it/s]

mean <x_i, x_j> =  0.0
mean absolute <x_i, x_j> =  0.2857142857142857
mean <f_i, f_j> =  -3.96508223080413e-18
mean absolute <f_i, f_j> =  0.040816326530612256 

mean <x_i, x_j> =  8.131516293641283e-18
mean absolute <x_i, x_j> =  0.2510201306431096
mean <f_i, f_j> =  -1.8973538018496328e-19
mean absolute <f_i, f_j> =  0.0313775163303887 

mean <x_i, x_j> =  1.1926223897340549e-17
mean absolute <x_i, x_j> =  0.2512695126549892
mean <f_i, f_j> =  -2.0328790734103208e-19
mean absolute <f_i, f_j> =  0.03140868908187365 

mean <x_i, x_j> =  -4.531522549490435e-17
mean absolute <x_i, x_j> =  0.2871500404188278
mean <f_i, f_j> =  3.96508223080413e-18
mean absolute <f_i, f_j> =  0.041021434345546816 



Label for linear: 100%|██████████| 4/4 [00:03<00:00,  1.20it/s]


mean <x_i, x_j> =  -2.0391851472706958e-17
mean absolute <x_i, x_j> =  0.3299310791114754
mean <f_i, f_j> =  5.664403186863043e-19
mean absolute <f_i, f_j> =  0.047133011301639355 

mean <x_i, x_j> =  4.531522549490435e-17
mean absolute <x_i, x_j> =  0.331740301171073
mean <f_i, f_j> =  1.1328806373726086e-18
mean absolute <f_i, f_j> =  0.047391471595867575 

mean <x_i, x_j> =  -5.936006894358137e-18
mean absolute <x_i, x_j> =  0.2901838653605915
mean <f_i, f_j> =  6.674619624363887e-19
mean absolute <f_i, f_j> =  0.036272983170073936 

mean <x_i, x_j> =  -1.1058862159352145e-17
mean absolute <x_i, x_j> =  0.30974780475562924
mean <f_i, f_j> =  1.8431436932253575e-18
mean absolute <f_i, f_j> =  0.038718475594453655 

mean <x_i, x_j> =  9.063045098980869e-18
mean absolute <x_i, x_j> =  0.3006656467464427
mean <f_i, f_j> =  2.5489814340883697e-18
mean absolute <f_i, f_j> =  0.042952235249491814 

mean <x_i, x_j> =  5.437827059388522e-17
mean absolute <x_i, x_j> =  0.2947882994845714
mean

...

mean <x_i, x_j> =  8.10976099540964e+27
mean absolute <x_i, x_j> =  8.10976099540964e+27
mean <f_i, f_j> =  141733920768.0
mean absolute <f_i, f_j> =  3.1045177130227146e+27 

mean <x_i, x_j> =  7.377326490722621e+31
mean absolute <x_i, x_j> =  7.377326490722621e+31
mean <f_i, f_j> =  -1470563143631182.2
mean absolute <f_i, f_j> =  3.0971865834817096e+31 

mean <x_i, x_j> =  9.440121544416869e+32
mean absolute <x_i, x_j> =  9.440121544416869e+32
mean <f_i, f_j> =  1.1764505149049458e+16
mean absolute <f_i, f_j> =  3.963199711763984e+32 

mean <x_i, x_j> =  2.089340718141283e+38
mean absolute <x_i, x_j> =  2.089340718141283e+38
mean <f_i, f_j> =  7.083549724304468e+21
mean absolute <f_i, f_j> =  7.998250828800432e+37 

mean <x_i, x_j> =  8.967382856642705e+34
mean absolute <x_i, x_j> =  8.967382856642705e+34
mean <f_i, f_j> =  -1.4411518807585587e+18
mean absolute <f_i, f_j> =  3.4328258084117515e+34 



Label for signature pde:  50%|█████     | 2/4 [00:09<00:09,  4.61s/it]

mean <x_i, x_j> =  3.9456650017406106e+36
mean absolute <x_i, x_j> =  3.9456650017406106e+36
mean <f_i, f_j> =  9.035139954469984e+18
mean absolute <f_i, f_j> =  1.656487249004345e+36 

mean <x_i, x_j> =  2.1326929209620783e+31
mean absolute <x_i, x_j> =  2.1326929209620783e+31
mean <f_i, f_j> =  884635641090633.1
mean absolute <f_i, f_j> =  8.952231963899503e+30 

mean <x_i, x_j> =  6.576281963438036e+30
mean absolute <x_i, x_j> =  6.576281963438036e+30
mean <f_i, f_j> =  -109143358316376.81
mean absolute <f_i, f_j> =  2.76088791709842e+30 

mean <x_i, x_j> =  2.673252804139919e+31
mean absolute <x_i, x_j> =  2.673252804139919e+31
mean <f_i, f_j> =  -96757023244288.0
mean absolute <f_i, f_j> =  8.97939311919643e+30 



/home/nikita/Code/kernel-timeseries-anomaly-detection/models/conformance.py:86: RuntimeWarning: overflow encountered in square
  d = d**2 / self.S[None, :] #shape (..., N, M)
/home/nikita/Code/kernel-timeseries-anomaly-detection/models/conformance.py:106: RuntimeWarning: overflow encountered in square
  d = d**2 / self.S #shape (..., M)
/home/nikita/Code/kernel-timeseries-anomaly-detection/models/conformance.py:86: RuntimeWarning: overflow encountered in square
  d = d**2 / self.S[None, :] #shape (..., N, M)
/home/nikita/Code/kernel-timeseries-anomaly-detection/models/conformance.py:106: RuntimeWarning: overflow encountered in square
  d = d**2 / self.S #shape (..., M)
Label for signature pde:  50%|█████     | 2/4 [00:13<00:13,  6.69s/it]


ValueError: Input contains infinity or a value too large for dtype('float64').

In [ ]:
# # Test GAK
# from experiment_code import normalize_streams
# def test_gak(dataset_name:str):
#     X_train, y_train, X_test, y_test = UCR_UEA_datasets().load_dataset(dataset_name)
#     unique_labels = np.unique(y_train)
#     num_classes = len(unique_labels)
#     N_train, T, d = X_train.shape

#     corpus, test = normalize_streams(X_train, X_test)
#     sigma=2
#     s = tslearn.metrics.sigma_gak(dataset=corpus,
#           n_samples=200,
#           random_state=0)  

#     for sigma in sorted([s, 0.0001, 0.1, 0.5, 1, 2, 5, 10, 20, 50, 100, 1000, 10000]):
#         uv = tslearn.metrics.cdist_gak(corpus[0:5]/20, corpus[0:5]/20, sigma=sigma)
#         print(uv, sigma, dataset_name)


# for dataset_name in [
#         #'ArticularyWordRecognition', 
#         #'BasicMotions', 
#         'Cricket', #only NaNs for all sigmas>1, else 0
#          ##########'ERing', #cant find dataset
#         #'Libras', 
#         #'NATOPS', 
#         #'RacketSports',     
#         #'FingerMovements', # estimates a bit low, 10e-3
#         #'Heartbeat',
#         'SelfRegulationSCP1',  #only NaNs for all sigmas>1, else 0       
#         #'UWaveGestureLibrary'
#         ]:
#     test_gak(dataset_name)

### Print CV results

In [ ]:
def average_labels(labelwise_dict:Dict[str, Dict[str, Any]],
                          field:str):
    """Averages the values of a field over the labels."""
    L = [param_dict[field] for param_dict in labelwise_dict.values()]
    min_len = min([len(Li) for Li in L])
    if min_len > 1:
        L = [Li[:min_len] for Li in L]
    return np.mean(L,axis=0)


def print_cv_tslearn_results(
        dataset_kernel_label_paramdict : Dict[str, Dict[str, Dict[str, Any]]],
        ):

    # return experiments
    for dataset_name, results in dataset_kernel_label_paramdict.items():
        print(dataset_name)
        kernelwise_dict = results["kernel_results"]
        n_classes = results['num_classes']
        ts_length = results['ts_length']
        n_train = results['N_train']
        path_dim = results['path dim']
        from experiment_code import print_dataset_stats
        print_dataset_stats(n_classes, path_dim, ts_length, n_train, "unknown")
        for kernel_name, labelwise_dict in kernelwise_dict.items():
            #final_auc_avgs = average_labels(labelwise_dict, "CV_train_auc")
            params_auc_avgs = average_labels(labelwise_dict, "auc_params")
            thresh_auc_avgs = average_labels(labelwise_dict, "auc_thresh")
            print(f"\n{kernel_name}")
            #print("final_auc_avgs", final_auc_avgs)
            print("params_auc_avgs", params_auc_avgs)
            print("thresh_auc_avgs", thresh_auc_avgs)
            if "truncated sig" in kernel_name:
                trunc_auc_avgs = average_labels(labelwise_dict, "auc_orders")
                print("orders_auc_avgs", trunc_auc_avgs)
        print("\nEnd dataset \n\n\n")

print_cv_tslearn_results(cv_best_models)

# (tslearn) Validate on Test

In [ ]:
def validate_tslearn(
        dataset_kernel_label_paramdict : Dict[str, Dict[str, Dict[str, Any]]],
        n_jobs:int = 1, 
        verbose:bool=True,
        ):
    """Validates the best models from cross validation on the
    tslearn datasets using kernel conformance scores."""
    experiments = {}
    for dataset_name, results in dataset_kernel_label_paramdict.items():

        # Load dataset
        print(dataset_name)
        X_train, y_train, X_test, y_test = UCR_UEA_datasets().load_dataset(dataset_name)
        unique_labels = np.unique(y_train)
        num_classes = len(unique_labels)
        N_train, T, d = X_train.shape
        N_test, _, _  = X_test.shape
        print_dataset_stats(num_classes, d, T, N_train, N_test)

        #validate on test set
        kernelwise_dict = results["kernel_results"]
        kernel_results = run_all_kernels(X_train, y_train, X_test, y_test, 
                            unique_labels, kernelwise_dict, fixed_length=True, 
                            n_jobs=n_jobs, verbose=verbose)
        experiments[dataset_name] = {"results": kernel_results, 
                                     "num_classes": num_classes, 
                                     "path dim":d,
                                     "ts_length":T, 
                                     "N_train":N_train, 
                                     "N_test":N_test}
    return experiments

test_results_tslearn = validate_tslearn(cv_best_models, n_jobs=8, verbose=False)

### Print test results

In [ ]:
def print_experiment_results(experiments, round_digits=5):
    for dataset_name, results in experiments.items():
        #Dataset:
        print("Dataset:", dataset_name)
        print_dataset_stats(results["num_classes"], results["path dim"], 
                            results["ts_length"], results["N_train"], 
                            results["N_test"])

        #Results for each kernel:
        for kernel_name, scores in results["results"].items():
            print("\nKernel:", kernel_name)
            print("Conformance AUC:", round(scores[0, 0], round_digits))
            print("Mahalanobis AUC:", round(scores[1, 0], round_digits))
            print("Conformance PR AUC:", round(scores[0, 1], round_digits))
            print("Mahalanobis PR AUC:", round(scores[1, 1], round_digits))

        print("\nEnd Dataset\n\n\n")


print_experiment_results(test_results_tslearn)

# Read CV data from file and validate

In [ ]:
#TODO